In [1]:
#from google.colab import drive
#drive.mount('/content/drive/')
#FOLDERNAME = './attempt2-main'
#assert FOLDERNAME is not None, "[!] Enter the foldername."

#import sys
#sys.path.append(f'/content/drive/My Drive/{FOLDERNAME}')
#%cd /content/drive/My\ Drive/$FOLDERNAME

In [2]:
import torch
from torch import nn
import numpy as np

from torch.utils.data import DataLoader

import time
import pickle
import math


from generatemidi import generate_midi
from model import Seq2SeqTransformer, JazzDataset, PositionalEncoding, TokenEmbedding

from torch.nn.functional import softmax
from torch import multinomial

/mmfs1/home/markpock/venvs/jupyter-notebook/lib64/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing altrepalt.py


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[1]
    tgt_seq_len = tgt.shape[1]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

    src_padding_mask = (src == pad_idx_har)
    tgt_padding_mask = (tgt == pad_idx_mel)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [5]:
# dress the data
BATCH_SIZE = 2

dataset = JazzDataset()

# Determine the size of the training set (e.g., 80% of the data)
train_size = int(0.8 * len(dataset))

# Determine the size of the validation set (e.g., 20% of the data)
valid_size = len(dataset) - train_size

# Split the DataLoader into training and validation sets
train, val = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_data = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
valid_data = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)

In uncompressed mode
loading data


In [6]:
SRC_VOCAB_SIZE = len(dataset.har_to_i) + 1
TGT_VOCAB_SIZE = len(dataset.mel_to_i) + 1
EMB_SIZE = 128
NHEAD = 8
FFN_HID_DIM = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
PAD_IDX = 1


pad_idx_har = dataset.har_to_i['end']
pad_idx_mel = dataset.mel_to_i[tuple(np.array([0,0]))]
start_idx_har = dataset.har_to_i['start']
start_idx_mel = dataset.mel_to_i[tuple(np.array([200.,0.]))]

In [7]:
model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

#optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)


In [8]:
# # training
# model.to(device)
# model.train()
# num_epochs = 100
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# print(f'Epoch 0 / {num_epochs}')
# l = len(train_data)
# for epoch in range(num_epochs):
#     losses = None
#     for i, batch in enumerate(train_data):
#         if i % 500 == 0 : print(f'E{epoch} Batch {i} / {l}')
#         optimizer.zero_grad()
#         harmony = batch['harmony'].to(device)
#         melody = batch['melody'].to(device)
#         melody_in = melody[:, :-1]

#         harmony_mask, melody_mask, harmony_padding_mask, melody_padding_mask = create_mask(harmony, melody_in)

#         logits = model(src=harmony, trg=melody_in, src_mask=harmony_mask, 
#                         tgt_mask=melody_mask, src_padding_mask=harmony_padding_mask,
#                         tgt_padding_mask=melody_padding_mask, memory_key_padding_mask=harmony_padding_mask)
#         optimizer.zero_grad()
#         melody_out = melody[:, 1:]
#         inp, tar = logits.reshape(-1, logits.shape[-1]), melody_out.reshape(-1)
       

#         loss = loss_fn(input=inp, target=tar)
#         loss.backward()
#         optimizer.step()
#         if i % 500 == 0 : print('loss: ', loss)
#         losses = loss
#     print(f'Epoch {epoch + 1} / {num_epochs}:\nLoss ↦ {loss}')
#     if ((epoch + 1) % 10) == 0:
#         print(f'Dumping modell at epoch {epoch + 1}')
#         with open(f'model_uncompressed_{epoch + 1}epochs.pkl', 'wb') as f: pickle.dump(model, f)

In [9]:
with open("model_uncompressed_60epochs.pkl", "rb") as f:
    model = pickle.load(f)

In [10]:
def beam_search(raw):
    # Got this off the interwebs. Works with numpy.
    # Eventual type of sequences is
    # list[list[float] (of length k, here 1), float]
    # I want dependent typing.
    
    data = softmax(raw, dim=-1).to('cpu').detach().numpy()
    sequences = [[list(), 0.0]]
    # walk over each step in sequence
    for row in data:
        all_candidates = list()
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score - np.log(row[j])]
                all_candidates.append(candidate)
        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select k best
        sequences = ordered[0]
    return sequences[0][0]

In [54]:
def greed(raw):
    '''
    Greedily takes the argmax over the raw probabilities.
    '''
    return torch.max(raw, dim=1)[1].item()

SIMPLE = [beam_search, greed]

def sample(raw):
    '''
    Applies softmax over the raw probabilities, then uses it
    to sample from a probability distribution.
    '''
    return multinomial(softmax(raw, dim=-1), 1).item()


def relu(raw):
    '''
    Applies relu, then samples.
    '''
    return sample(nn.functional.relu(raw))

def leaky(raw):
    '''
    Applies leaky relu, then samples.
    '''
    return sample(nn.functional.leaky_relu(raw, 0.1))

def leaky2(raw):
    '''
    Applies leaky relu, then samples.
    '''
    return sample(nn.functional.leaky_relu(raw, 0.5))

LINEAR = [sample, relu, leaky, leaky2]

# The idea below is to sharpen the probabilities the slightest bit
# to get more consistency. First we have x sqrt(x), which is a
# little too repetitive and rhythmically constrained, then we have
# x ln(x), which is asymptotically lower and should prove to possibly
# work out a touch better...

def xsqrtx(raw):
    return sample(raw * torch.sqrt(nn.functional.relu(raw)))

def xlnx(raw):
    # Needs the extra relu bc log can sometimes be negative?
    return sample(raw * torch.log(nn.functional.relu(raw) + 1))

def exp(raw):
    # This may be stupid. Scratch that, it probably is. When in
    # doubt, try? Heh.
    return sample(torch.exp(raw))

def exp(raw):
    return sample(raw * raw)

SHARPEN = [xsqrtx, xlnx, exp]

def tanh(raw):
    return sample(torch.tanh(raw))

def sigmoid(raw):
    return sample(torch.sigmoid(raw))

def cuberoot(raw):
    # Must be declared piecewise to support negative numbers
    relued = nn.functional.relu(raw)
    pos = torch.pow(relued, 1/3)
    neg = torch.pow(relued - raw, 1/3)
    return sample(pos - neg)

SMOOTH = [tanh, sigmoid, cuberoot]

def xsqrtx_tanh(raw):
    relued = nn.functional.relu(raw)
    pos = raw * torch.sqrt(relued)
    neg = torch.tanh(relued - raw)
    return sample(pos - neg)

def xsqrtx_cbrt(raw):
    relued = nn.functional.relu(raw)
    pos = raw * torch.sqrt(relued)
    neg = torch.pow(relued - raw, 1/3)
    return sample(pos - neg)

PIECEWISE = [xsqrtx_tanh, xsqrtx_cbrt]

# In the vein of sharpening the probabilities, we now explore
# a slightly greedier approach using top-k sorting.
# Three variations follow:
# - Apply softmax to the top-k values, zeroing out all other
#   probabilities.
# - Apply top-k and zero out all other raw values, then take
#   softmax.
# - Uniformly choose from among the top-k indices.

# We must also account for precisely how much of the data
# we wish to choose. We start arbitrarily by using the top
# tenth of the probabilities, about 60 words.

def harsh_topk(raw):
    K = int(raw.shape[1]) // 10
    values, indices = torch.topk(raw, K, dim=-1)
    zeros = torch.zeros_like(raw)
    zeros[:, indices[0]] = softmax(values, dim=-1)
    return multinomial(zeros, 1).item()

def lenient_topk(raw):
    K = int(raw.shape[1]) // 10
    values, indices = torch.topk(raw, K, dim=-1)
    zeros = torch.zeros_like(raw)
    zeros[:, indices[0]] = values
    return multinomial(softmax(zeros, dim=-1), 1).item()

TOPK = [harsh_topk, lenient_topk]

# Will probably want to pare K to be lower in here
def uniform_topk(raw, K):
    values, indices = torch.topk(raw, K, dim=-1)
    index = torch.randint(0, indices.shape[1], (1,)).item()
    return indices[0, index]

def uniform_topk10(raw):
    # 1 tenth
    return uniform_topk(raw, int(raw.shape[1]) // 10)

def uniform_topk20(raw):
    # 1 20th
    return uniform_topk(raw, int(raw.shape[1]) // 20)

def uniform_topk30(raw):
    # 1 30th
    return uniform_topk(raw, int(raw.shape[1]) // 30)

def uniform_topk40(raw):
    # 1 30th
    return uniform_topk(raw, int(raw.shape[1]) // 40)

def uniform_topk50(raw):
    # 1 30th
    return uniform_topk(raw, int(raw.shape[1]) // 40)

UNIFORM = [uniform_topk10, uniform_topk20, uniform_topk30, uniform_topk40, uniform_topk50]


def xsqrtx_topk(raw):
    K = int(raw.shape[1]) // 20
    values, indices = torch.topk(raw, K, dim=-1)
    
    applied = values * torch.sqrt(nn.functional.relu(values))
    
    softmaxed = softmax(applied, dim=-1)
    index = multinomial(softmaxed, 1).item()
    return indices[0, index]

def xlnx_topk(raw):
    K = int(raw.shape[1]) // 20
    values, indices = torch.topk(raw, K, dim=-1)

    
    applied = values * torch.log(nn.functional.relu(values) + 1)
    
    softmaxed = softmax(applied, dim=-1)
    index = multinomial(softmaxed, 1).item()
    return indices[0, index]
                      
def tanh_topk(raw):
    K = int(raw.shape[1]) // 20
    values, indices = torch.topk(raw, K, dim=-1)
    
    applied = torch.tanh(values)
    
    softmaxed = softmax(applied, dim=-1)
    index = multinomial(softmaxed, 1).item()
    return indices[0, index]

def sigmoid_topk(raw):
    K = int(raw.shape[1]) // 20
    values, indices = torch.topk(raw, K, dim=-1)
    
    applied = torch.sigmoid(values)
    
    softmaxed = softmax(applied, dim=-1)
    index = multinomial(softmaxed, 1).item()
    return indices[0, index]

APPLIED = [xsqrtx_topk, xlnx_topk, tanh_topk, sigmoid_topk]

FUNCTIONS = SIMPLE + LINEAR + SHARPEN + SMOOTH + PIECEWISE + TOPK + UNIFORM + APPLIED

In [55]:
# function to generate output sequence using a provided algorithm
# for the next word

def decode(model, src, src_mask, max_len, start_symbol, nextword=greed):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask).to(device)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len - 1):
        tgt_mask = (generate_square_subsequent_mask(ys.size(1))
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)

        prob = model.generator(out[:, -1, :])
        
        next_word = nextword(prob)

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == pad_idx_mel or next_word == start_idx_mel:
            print('END REACHED')
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str, func):
    model.eval()
    src = src_sentence
    num_tokens = src.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=start_idx_mel, nextword=func).flatten()
    return tgt_tokens

In [56]:
fnames = lambda fns: {f.__name__ : f for f in fns}

def indiv(harmony, func, filename='test', name='greed'):
    out = translate(model, harmony.to(device).unsqueeze(dim=0), func)
    length = out.shape[0]

    inferred = np.zeros((length, 2))
    for i in range(length):
        #if (int(out[i])) == pad_idx_mel or\
        #    (int(out[i])) == start_idx_mel: break
        inferred[i] = dataset.i_to_mel[int(out[i])]
    generate_midi(inferred[:-1], f'{filename}_model_{name}')

def workflow(batch, filename='test', gt=False, functions=FUNCTIONS):
    melody, harmony = batch['melody'], batch['harmony']
    
    for name, func in fnames(functions).items():
        print(name)
        indiv(harmony, func, filename, name)
    
    if not gt: return
    
    gt = np.zeros((melody.shape[0], 2))
    for i in range(1, melody.shape[0] - 2):
        res = int(melody[i - 1].item())
        breaker = int(melody[i].item())
        if (breaker == start_idx_mel or
            breaker == pad_idx_mel):
            break
        gt[i] = dataset.i_to_mel[res]

    generate_midi(gt[1:-1, :], f'{filename}_gt')


In [38]:
workflow(val[0], filename='./topk/60epochs', gt=True,
        functions=UNIFORM + APPLIED)

uniform_topk10
uniform_topk20
END REACHED
uniform_topk30
uniform_topk40
uniform_topk50
xsqrtx_topk
xlnx_topk
tanh_topk
END REACHED
sigmoid_topk


In [39]:
RCHANGES    = np.array(["Bbj7", "Bbj7", "G7", "G7", "C-7", "C-7", "F7", "F7", # A1
                        "Bbj7", "Bbj7", "G7", "G7", "C-7", "C-7", "F7", "F7", 
                        "F-7", "F-7", "Bb7", "Bb7", "Ebj7", "Ebj7", "Ab7", "Ab7", 
                        "D-7", "D-7", "G7", "G7", "C-7", "C-7", "F7", "F7",
                        "Bbj7", "Bbj7", "G7", "G7", "C-7", "C-7", "F7", "F7", # A2
                        "Bbj7", "Bbj7", "G7", "G7", "C-7", "C-7", "F7", "F7", 
                        "F-7", "F-7", "Bb7", "Bb7", "Ebj7", "Ebj7", "Ab7", "Ab7", 
                        "D-7", "D-7", "G7", "G7", "C-7", "C-7", "F7", "F7",
                        "D7", "D7", "D7", "D7", "D7", "D7", "D7", "D7", # B
                        "G7", "G7", "G7", "G7", "G7", "G7", "G7", "G7", 
                        "C7", "C7", "C7", "C7", "C7", "C7", "C7", "C7", 
                        "F7", "F7", "F7", "F7", "F7", "F7", "F7", "F7",
                        "Bbj7", "Bbj7", "G7", "G7", "C-7", "C-7", "F7", "F7", # A3
                        "Bbj7", "Bbj7", "G7", "G7", "C-7", "C-7", "F7", "F7", 
                        "F-7", "F-7", "Bb7", "Bb7", "Ebj7", "Ebj7", "Ab7", "Ab7", 
                        "C-7", "C-7", "F7", "F7", "Bbj7", "Bbj7", "Bbj7", "Bbj7"
                        ])


In [40]:


def inside_outside_metric_rc(notes, start_offset=0, chorus_count=1):
    """
    - harmony: (l, 1) 
    - notes: (l2, 2)
    - returns inside_outside_metric for rhythm changes in Bb
    """
    harmony = RCHANGES
    song_length = 128
    harmony = np.tile(harmony, 4)
    # print(notes.shape)
    onsets = np.floor(np.roll(np.cumsum(notes[:, 1]), shift=1))# beats in which notes begin
    
    onsets[0] = 0
    # print("N", notes[:20, :])
    # print(onsets[:20])
    befores = np.where(onsets < start_offset)
    onsets = np.delete(onsets, befores) - start_offset
    if np.array([befores]).size > 0:
        notes = notes[np.max(befores)+1:, :]
    # notes = np.delete(notes, befores)
    # onsets = onsets[i:] #- start_offset
    extras = np.where(onsets >= song_length * chorus_count)
    # print("e", notes[extras, :])
    # print(onsets[extras])
    onsets = np.delete(onsets, extras)
    #print(notes[:10])
    #print(onsets[:10])
    #print(onsets[:10])
    notes[:, 1] = notes[:, 1]
    chord_profiles = {
        "Bbj7": [38, 41, 45, 46, 50, 53, 57, 58, 62, 65, 69, 70, 74, 77, 81, 82, 86, 89, 93, 94],
        "G7": [38, 50, 62, 74, 86, 41, 53, 65, 77, 89, 43, 55, 67, 79, 91, 47, 59, 71, 83, 95], 
        "C-7": [36, 48, 60, 72, 84, 96, 39, 51, 63, 75, 87, 43, 55, 67, 79, 91, 46, 58, 70, 82, 94], #CEbGBb
        "F7": [41, 53, 65, 77, 89, 45, 57, 69, 81, 93, 39, 51, 63, 75, 87, 36, 48, 60, 72, 84, 96], # FAEbC
        "F-7": [41, 53, 65, 77, 89, 44, 56, 68, 80, 92, 39, 51, 63, 75, 87, 36, 48, 60, 72, 84, 96],
        "Bb7": [38, 41, 44, 46, 50, 53, 56, 58, 62, 65, 68, 70, 74, 77, 80, 82, 86, 89, 92, 94],
        "Ebj7": [38, 50, 62, 74, 86, 39, 51, 63, 75, 87, 43, 55, 67, 79, 91, 46, 58, 70, 82, 94], #DEbGBb
        "Ab7": [44, 56, 68, 80, 92, 39, 51, 63, 75, 87, 36, 48, 60, 72, 84, 96, 42, 54, 66, 78, 90],
        "D-7": [41, 53, 65, 77, 89, 45, 57, 69, 81, 93, 38, 50, 62, 74, 86, 36, 48, 60, 72, 84, 96], # FADC
        "D7": [42, 54, 66, 78, 90, 46, 58, 70, 82, 94, 38, 50, 62, 74, 86, 36, 48, 60, 72, 84, 96], 
        "C7": [36, 48, 60, 72, 84, 96, 40, 52, 64, 76, 88, 43, 55, 67, 79, 91, 46, 58, 70, 82, 94], 
    }
    num_outside = 0
    num_inside = 0
    cur_beat = 0
    # i = 0
    # while cur_beat < 128+start_offset:
    #     if notes[i, 0] == 0 or start_offset > cur_beat:
    #         print("zero", cur_beat, notes[i, 0], notes[i, 1])
    #         cur_beat += notes[i, 1]
    #         i += 1
    #         continue
    #     curr_chord = harmony[math.floor(cur_beat) - start_offset]
    #     curr_note = notes[i, 0]
    #     good_notes = chord_profiles[curr_chord]
    #     if curr_note in good_notes:
    #         print("YES", curr_note, curr_chord, cur_beat)
    #         num_inside += 1
    #     else:
    #         print("NO", curr_note, curr_chord, cur_beat)
    #     cur_beat += notes[i, 1]
        # i += 1
        
        
    # print(harmony.shape)
    #print(np.max(onsets))
    for i in range(onsets.shape[0]):
        if notes[i, 0] == 0:
            continue
        curr_chord = harmony[int(onsets[i])]
        # print(onsets[i])
        good_notes = chord_profiles[curr_chord]
        curr_pitch = int(notes[i, 0])
        # print(curr_chord, curr_pitch, onsets[i] % 4, np.floor(onsets[i]/4))
        if curr_pitch in good_notes:
            #print("YES", curr_pitch, curr_chord, onsets[i])
            num_inside += 1
        else:
            #print("NO", curr_pitch, curr_chord, onsets[i])
            num_outside += 1

    return num_inside / notes.shape[0]

In [41]:
harm = [dataset.har_to_i[chord] for chord in (['start'] + list(RCHANGES) + ['end'])]
harm = torch.tensor(harm)
harm


tensor([721, 217, 217, 633, 633, 254, 254, 573, 573, 217, 217, 633, 633, 254,
        254, 573, 573, 554, 554, 201, 201, 517, 517,  81,  81, 314, 314, 633,
        633, 254, 254, 573, 573, 217, 217, 633, 633, 254, 254, 573, 573, 217,
        217, 633, 633, 254, 254, 573, 573, 554, 554, 201, 201, 517, 517,  81,
         81, 314, 314, 633, 633, 254, 254, 573, 573, 333, 333, 333, 333, 333,
        333, 333, 333, 633, 633, 633, 633, 633, 633, 633, 633, 273, 273, 273,
        273, 273, 273, 273, 273, 573, 573, 573, 573, 573, 573, 573, 573, 217,
        217, 633, 633, 254, 254, 573, 573, 217, 217, 633, 633, 254, 254, 573,
        573, 554, 554, 201, 201, 517, 517,  81,  81, 254, 254, 573, 573, 217,
        217, 217, 217, 723])

In [57]:
def eval_indiv(harmony, func):
    out = translate(model, harmony.to(device).unsqueeze(dim=0), func)
    length = out.shape[0]

    inferred = np.zeros((length, 2))
    for i in range(length):
        inferred[i] = dataset.i_to_mel[int(out[i])]
    
    return inside_outside_metric_rc(inferred)

def evalflow(harmony, functions=FUNCTIONS):
    metric = {}
    for name, func in fnames(functions).items():
        metric[name] = []
        for _ in range(10):
            num = eval_indiv(harmony, func)
            metric[name].append(num)
        #print(f'{name} ↦ {sum(metric[name]) / len(metric[name]):.3f}')
    return metric

In [52]:
def gen_indiv(harmony, func):
    out = translate(model, harmony.to(device).unsqueeze(dim=0), func)
    length = out.shape[0]

    inferred = np.zeros((length, 2))
    for i in range(length):
        inferred[i] = dataset.i_to_mel[int(out[i])]
    
    generate_midi(inferred[1:-1], 'inferred')

gen_indiv(harm, lenient_topk)

In [58]:
end = evalflow(harm)


beam_search ↦ 0.230
greed ↦ 0.230
sample ↦ 0.216
relu ↦ 0.184
leaky ↦ 0.230
leaky2 ↦ 0.210
xsqrtx ↦ 0.256
xlnx ↦ 0.241
exp ↦ 0.007
END REACHED
tanh ↦ 0.124
sigmoid ↦ 0.096
END REACHED
cuberoot ↦ 0.205
xsqrtx_tanh ↦ 0.253
xsqrtx_cbrt ↦ 0.256
harsh_topk ↦ 0.217
lenient_topk ↦ 0.153
uniform_topk10 ↦ 0.274
uniform_topk20 ↦ 0.283
uniform_topk30 ↦ 0.279
uniform_topk40 ↦ 0.292
uniform_topk50 ↦ 0.288
xsqrtx_topk ↦ 0.244
xlnx_topk ↦ 0.229
tanh_topk ↦ 0.279
sigmoid_topk ↦ 0.295


NameError: name 'metric' is not defined

In [59]:
for name in end:
    print(f'{name} ↦ {sum(end[name]) / len(end[name]):.4f}')

beam_search ↦ 0.2296
greed ↦ 0.2296
sample ↦ 0.2163
relu ↦ 0.1844
leaky ↦ 0.2296
leaky2 ↦ 0.2104
xsqrtx ↦ 0.2563
xlnx ↦ 0.2407
exp ↦ 0.0074
tanh ↦ 0.1239
sigmoid ↦ 0.0963
cuberoot ↦ 0.2047
xsqrtx_tanh ↦ 0.2526
xsqrtx_cbrt ↦ 0.2563
harsh_topk ↦ 0.2170
lenient_topk ↦ 0.1526
uniform_topk10 ↦ 0.2741
uniform_topk20 ↦ 0.2830
uniform_topk30 ↦ 0.2793
uniform_topk40 ↦ 0.2919
uniform_topk50 ↦ 0.2881
xsqrtx_topk ↦ 0.2437
xlnx_topk ↦ 0.2289
tanh_topk ↦ 0.2793
sigmoid_topk ↦ 0.2948
